## Convert NDEx network to Neo4j ##

To do this tutorial you need to have Neo4j installed and running.  Also pip install ndex2 and neo4jrestclient.

In [1]:
from neo4jrestclient.client import GraphDatabase
from neo4jrestclient import client
import ndex2

In [6]:
niceCx = ndex2.create_nice_cx_from_server(server='public.ndexbio.org',
                                          uuid='c66cc833-6194-11e5-8ac5-06603eb7f303')

neo4j_node_lookup = {}
db = GraphDatabase("http://localhost:7474", username="neo4j", password="mypassword")

# CREATE NODES
gene = db.labels.create("Gene")
for k, v in niceCx.get_nodes():
    # Keep track of the neo4j nodes that were added
    # so they can be used to create the edges
    neo4j_node_lookup[v.get_id()] = db.nodes.create(name=v.get_name())
    gene.add(neo4j_node_lookup[v.get_id()])

# CREATE EDGES
for k, v in niceCx.get_edges():
    # Retrieve the nodes from the look up and create
    # the relationship
    g1 = neo4j_node_lookup.get(v.get_source())
    g2 = neo4j_node_lookup.get(v.get_target())
    g1.relationships.create(v.get_interaction(), g2)


In [12]:
# Enter any Neo4j cypher query here    
q = 'MATCH (n)-[p]-(r) RETURN n,p,r'
    
results = db.query(q, returns=(client.Node, str, client.Node))
print(len(results))

70
